# Circuit simulations

Every gdsfactory component can access its netlist `get_netlist()` and Sparameters.

You can use a circuit simulation tool such as [Simphony](https://github.com/BYUCamachoLab/simphony)

You can install simphony with `pip install simphony` and interface with the circuit models and Sparameters stored in [gdslib](https://github.com/gdsfactory/gdslib).

`pip install gdslib`

In [ ]:
import pp

c = pp.c.mzi()
pp.qp(c)

n = c.get_netlist()

`gdslib` has also an MZI model. However we can make sure we get the exact cells from the component

In [ ]:
import gdslib as gl

m = gl.mzi()
gl.sweep_simulation(m)

In [ ]:
from simphony.netlist import Subcircuit
from gdslib import sweep_simulation

component = pp.c.mzi()
circuit = Subcircuit(component.name)
n = component.get_netlist()

In [ ]:
model_name_tuple = []

for i in n.instances.keys():
    component_type = n.instances[i]['component']
    component_settings = n.instances[i]['settings']
    c = pp.c.component_type2factory[component_type]()
    # c = pp.c.component_type2factory[component_type](**component_settings)  # needs fix
    model = gl.load(c)
    model_name_tuple.append((model, i))
    
e = circuit.add(model_name_tuple)

In [ ]:
for k, v in n.connections.items():
    c1, p1 = k.split(',')
    c2, p2 = v.split(',')
    circuit.connect(c1, p1, c2, p2)

In [ ]:
circuit.elements["mmi1x2_12_0"].pins["W0"] = "input"
circuit.elements["mmi1x2_88_0"].pins["W0"] = "output"

sweep_simulation(circuit)

In [ ]:
n.instances[i]['settings']